# Parsing Youtube Data Structures into a Schema_df

 The purpose of his jupyter notebook is to parse the collected data structures in earlier iterations of utilising the data donation tool into a schema_df which can be used to inform future iterations of the data donation tool. Note that there are two notebooks to parse the Youtube data structures in the repository as the Youtube data takeout contains both CSV and JSON files. 

In [41]:
import pandas as pd
import json
import numpy as np
from pathlib import Path  


main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/"


## Creating functions
### process_col_path()
The 'process_col_path()' function checks whether the value in a row of for one of the column paths is actually a datatype and stores this value in a column data_type and replaces the original value with NA. The data types are the lowest level values in the JSON files.

In [42]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object', 'str', 'int', 'float', 'bool', 'dict', 'list']

#Define the column names
columns = ['col_path_1','col_path_2','col_path_3','col_path_4', 'col_path_5']



# Define the function 'process_col_path()'
def process_col_path(row, columns, data_types):

    """
    row: Rows in the dataframe
    columns: List of column names of column path columns 
    data_types: List of the values that are data types
    """

    row['data_type'] = ''
    for column in columns:

        #If the value stored in the column is found in the list 'data_types', 
        if row[column] in data_types:
            # this value is placed in the column 'data_type'
            row['data_type'] = row[column]
            # and the original value is replaced with NA
            row[column] = np.nan

        #If the value is not found in the 'data_types' list, the original value is returned
        else:
            row[column]
    return row


### file_paths()
The 'file_paths()' function splits up the paths to where the JSON file is stored in the folder and provides the name of the json file. 
- 'path_{1,2,3,4}': Column including the names of the {first, second, third, fourth} level folder where the JSON file is stored
- 'json_name': Column including the name of the JSON file

If the JSON name appears in the 'path_{1,2,3,4}' column, this name is replaced with NA and stored in the column 'json_name'

In [43]:
def file_paths(df):
    
    # Create different columns for each part of the document path
    df['path_1'] = df['variable'].str.split('/', n=1).str[0]
    df['path_2'] = df['variable'].str.split('/', n=3).str[1]
    df['path_3'] = df['variable'].str.split('/', n=3).str[2]
    df['path_4'] = df['variable'].str.split('/', n=3).str[3]

    print(df['variable'])

    # Create a column with the JSON name
    df['json_name'] = df['variable'].str.rsplit('/', n=1).str[-1]


    # As the JSON name is stored in the json_name column, fill other parts of the path with Na if the name of the JSON is present
    # If the value is not the name of the value, return the original value
    mark = ".json"

    df[['path_2', 'path_3', 'path_4']] =  df[['path_2', 'path_3', 'path_4']].map(lambda x: np.nan if isinstance(x, str) and mark in x else x)

    # Unlist the value column (where the JSON info is stored) if it contains a list, otherwise return the original value 
    df['value'] = df['value'].apply(lambda x: x[0] if isinstance(x, list) else x)

    return df

### detect_list()

In [44]:
# Assign list if the data in the original structure is a list
    
def detect_list(x):
    # If the data type of the value is list, 'LIST' is assigned in the '_LIST' columns (see'column_paths())
    if isinstance(x, list):
        return 'LIST'
    # If the value is missing, 'MISSING' is assigned
    elif pd.isna(x):
        return 'MISSING'
    # If the value is 'No data' (there is an empty place holder) 'MISSING' is assigned
    elif x == 'No data':
            return 'MISSING'
    # if the value is not missing and is not a list, 'NO LIST' is assigned
    else:
        return 'NO LIST'

### columns_paths()
Through this function, we map the json paths to each value by putting the keys in separate columns: colpath_{1,2,3,4,5} (see 'row_column_paths()')

In [45]:
def column_paths(df, col_var, get_var, unlist_var, explode_var):

        # Extract nested values using keys
        """
        1. Selects the key obtained in the previous iteration stored in 'get_var'
        2. The key put into a get() function which extracts values of a dictionary based on the keys
        3. The get() function extracts the values from the (nested) dictionary stored in 'col_var'
        4. If the get() function fails, None is returned
        5. The steps above are only performed if the value stored in 'col_var' is a dictionary, otherwise None is returned
        6. All steps above are executed for each row in the df
        """
        df[unlist_var] = df.apply(lambda row: row[col_var].get(row[get_var], None) if isinstance (row[col_var], dict) else None, axis=1)

        
         # For each each column_path, check if the value contains a list
        df[f'{explode_var}_LIST'] = df[unlist_var].apply(detect_list)

        df[unlist_var] = df[unlist_var].apply(
        lambda x: [x] if not isinstance(x, list) else x)

        df = df.explode(unlist_var)
        

        # Unlist data to avoid double nested lists 
        df[unlist_var] = df[unlist_var].apply(lambda x: x[0] if isinstance(x, list) else x)

        df[explode_var] = df[unlist_var]

         # Explode the list containing the obtained key values into one key per row
        df = df.explode(explode_var)

    
        
        return df

### row_column_paths()
This function prepares the row paths and consequently executes the column_paths function to obtain the paths of keys (stored in individual columns) to reach the lowest value which is where the actual data is stored


In [46]:
def row_column_paths(df):
    # Initialize the new columns
    df['row_path'] = '' 
    df['col_path_1'] = ''  
    df['col_path_2'] = ''  
    df['col_path_3'] = ''
    df['col_path_4'] = ''
    df['col_path_5'] = '' 

   
    # Take all the keys of the dictionaries in the 'value' column and wrap them in a list
    df['row_path'] = df['value'].apply(lambda x: list(x.keys()))


    
    # Take the level 1 keys stored in a list and store them in individual rows
    df = df.explode('row_path')
  


    # Initiate the list of columns needed to inform the function column_paths()
    col_var = ['value', 'col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values']
    get_var = ['row_path', 'col_path_1', 'col_path_2', 'col_path_3', 'col_path_4']
    unlist_var = ['col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values', 'col_path_5_values']
    explode_var = ['col_path_1', 'col_path_2', 'col_path_3', 'col_path_4', 'col_path_5']

    # Execute the colums_path() function
    # zip is necessary due to the large number of variables needed in the column_paths function
    for r, g, u, e in zip(col_var, get_var, unlist_var, explode_var):
        df = column_paths(df, r, g, u, e)
        col_var = unlist_var

    return df


### clean_and_store()
This function orders the columns in the DataFrame, resets the index, fills the NA and saves the DataFrame as CSV

In [47]:
def clean_and_store(df, file_name):
 
    # Reorder the columns in the df
    df = df[['variable', 'value', 'path_1', 'path_2',
        'path_3', 'path_4', 'json_name', 'row_path', 
        'col_path_1', 'col_path_1_LIST',
                     'col_path_2', 'col_path_2_LIST',
                     'col_path_3', 'col_path_3_LIST','data_type']]
    
    # Reset the index
    df.reset_index(drop=True, inplace=True)

    # fill na values with 'Missing'
    df = df.fillna('Missing')

    # Save the DataFrame 
    df.to_csv(f"{main_path}Youtube/Output/Output_" + file_name + '.csv', index=False)
    
    return df

### structure_donations()
The structure_donations() function executes all functions above and results in a saved DataFrame for each data structure.


In [48]:
def structure_donations(data):

    """
    data: The unprocessed data structure JSON that will be processed
    """

    # Store the path to the data structure
    data = Path(data)  
    
    # Save teh file name of the data structure
    file_name = Path(data).stem 

    # Load JSON file (data structures)
    with open(data, 'r') as f:
        data = json.load(f)

    
    # Flatten JSON (handling nested structures)
    df = pd.json_normalize(data, max_level=0)
    
    # Extract column names
    cols = df.columns.tolist()


    # From wide to long df
    df = pd.melt(df, value_vars= cols)

    # Execute the 'file_paths()' function and store the result in df
    df = file_paths(df)
    
    # Execute the 'row_column_paths()' function and store the result in df
    df = row_column_paths(df)
    
    # Execute the 'process_col_path()' function and store the result in df
    df = df.apply(lambda row: process_col_path(row, columns, data_types), axis=1)

    # Execute the 'clean_and_store()' function and store the result in df
    df = clean_and_store(df, file_name)
    
    
    return df


## Execute 'structure_donations()': Transform data structures from JSON format to tabular format

In [50]:

input_directory = Path(f'{main_path}Youtube/Input_test')  
print(input_directory)


/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/Youtube/Input_test


In [51]:
# Execute the 'structure_donations()' function for each file (data structure) in the input directory
for file in input_directory.iterdir():  
    if file.is_file():  
        structure_donations(file)
        

0          historial/historial-de-búsqueda.json
1    historial/historial-de-reproducciones.json
Name: variable, dtype: object
0    geschiedenis/kijkgeschiedenis.json
1    geschiedenis/zoekgeschiedenis.json
Name: variable, dtype: object
0     youtube_takeout/history/watch-history.json
1    youtube_takeout/history/search-history.json
Name: variable, dtype: object
0     istorija/paieškos istorija.json
1    istorija/žiūrėjimo istorija.json
Name: variable, dtype: object
0    history/search-history.json
1     history/watch-history.json
Name: variable, dtype: object
0      istoric/istoricul căutărilor.json
1    istoric/istoricul-vizionărilor.json
Name: variable, dtype: object
0     youtube_takeout/history/watch-history.json
1    youtube_takeout/history/search-history.json
Name: variable, dtype: object


## Merge all data structures into one schema_df

In [52]:
# Path to the folder containing CSV files
output_path = f"{main_path}Youtube/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames
merged_df = pd.concat(dfs, axis=0, ignore_index=True)

col_subset = merged_df.columns.tolist()
col_subset.remove('value')

# Drop rows that are completely identical across all columns
merged_df = merged_df.drop_duplicates(subset= col_subset)

# Replace 'Missing' with NA
merged_df = merged_df.replace('Missing', np.nan)


/tmp/ipykernel_812641/2804539798.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df = merged_df.replace('Missing', np.nan)


### extract_path()

In [53]:
max_col_path = 3
def extract_path(df, max_col_path) -> tuple[str, ...]:
    path = []
    for col in ["row_path"] + [f"col_path_{i}" for i in range(1, max_col_path)]:
        val = df[col]
        if pd.notna(val) and str(val) != "Missing":
            path.append(str(val).strip())
    return tuple(path)

In [54]:

merged_df['path'] = merged_df.apply(lambda x: extract_path(x, max_col_path), axis = 1)

### Clean up and reduce number of columns

In [55]:
def list_path(row):
    path = row['path'] 
    for i in range(len(path)):
        if row[f"col_path_{i+1}_LIST"] == "LIST":
            var_type = 'list'
            list_path = json.dumps(path[:i+2])
            subfield_path = path[-1]
            column_name = subfield_path
            subfield_path = json.dumps(subfield_path)
        elif row[f"col_path_{i+1}_LIST"] == "NO LIST":
            var_type = 'static'
            list_path = np.nan
            subfield_path = json.dumps(path)
            column_name = path[-1]
        elif row[f"col_path_{i+1}_LIST"] == "MISSING" :
            var_type = 'skip'
            list_path = np.nan
            subfield_path = json.dumps(path)
            column_name = path[-1]
        return list_path, subfield_path, column_name, var_type


merged_df[['list_path', 'subfield_path', 'column_name', 'var_type']] = merged_df.apply(lambda x: pd.Series(list_path(x)), axis = 1)



### id creation

In [56]:

merged_df['name'] = merged_df['json_name'].str.replace(".json", "")

id_df = merged_df[['name', 'path']]
id_df = id_df.drop_duplicates()
id_df['id'] = ''

for n in range(1,max_col_path):
    duplicates = id_df[id_df.duplicated(subset='id', keep=False)]

    for i, row in duplicates.iterrows():
        path = list(row['path'])
        path_zero = path[0]
        path_rest = path[-n:]  # Varies each loop to attempt uniqueness
        name = row['name'] 

        if [path_zero] == path_rest:
            id_list = [name] + path_rest
        else:
            id_list = [name]+ [path_zero] + path_rest

    
        new_id = ':'.join(id_list)

        id_df.at[i, 'id'] = new_id  # Properly update the DataFrame

merged_df = pd.merge(merged_df, id_df, on = ['name', 'path'], how = 'left')

col = merged_df.pop('id') 
merged_df.insert(0, 'id', col) 
#merged_df['id']= merged_df['id'].fillna(merged_df['variable'])


In [57]:
dup = merged_df[merged_df.duplicated('id', keep= False)]
dup['duplicate_flag'] = 'Yes'
dup = dup[['id', 'duplicate_flag']]
merged_df = pd.merge(merged_df, dup, on = 'id', how = 'left')
merged_df['duplicate_flag'] = merged_df['duplicate_flag'].fillna(value = 'No')
merged_df = merged_df.drop_duplicates()

/tmp/ipykernel_812641/1499538199.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dup['duplicate_flag'] = 'Yes'


In [58]:
# Save the final merged DataFrame
merged_df.to_csv(f"{main_path}Youtube/Final/Merged_structures_YT.csv", index=False)